# ReAct Agent

Adapted from https://github.com/opendatahub-io/llama-stack-demos/blob/main/demos/rag_agentic/notebooks/Level3_advance_agentic_with_Prompt_Chaining_ReAct.ipynb

In [1]:
import rich
import uuid
from llama_stack_client import Agent, LlamaStackClient, AgentEventLogger
from llama_stack_client.lib.agents.client_tool import client_tool
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput

In [2]:
# this may be missing from pyproject.toml, add it if you need it
#!uv pip install geocoder

In [3]:
@client_tool
def get_location(query: str = "location"):
    """
    Provide the location upon request.

    :param query: The query from user
    :returns: Information about user location
    """
    import geocoder
    
    try:
        g = geocoder.ip('me')
        if g.ok:
            return f"Your current location is: {g.city}, {g.state}, {g.country}" # can be modified to return latitude and longitude if needed
        else:
            return "Unable to determine your location"
    except Exception as e:
        return f"Error getting location: {str(e)}"

In [4]:
client = LlamaStackClient(
    base_url="http://localhost:8321",
)

In [5]:
models = client.models.list()
rich.print(models)

[
    Model(
        identifier='meta-llama/Llama-3.2-3B-Instruct',
        metadata={},
        api_model_type='llm',
        provider_id='ollama',
        provider_resource_id='llama3.2:3b-instruct-fp16',
        type='model',
        model_type='llm'
    ),
    Model(
        identifier='all-MiniLM-L6-v2',
        metadata={'embedding_dimension': 384.0},
        api_model_type='embedding',
        provider_id='ollama',
        provider_resource_id='all-minilm:latest',
        type='model',
        model_type='embedding'
    )
]

In [6]:
toolgroups = client.toolgroups.list()
rich.print(toolgroups)

[
    ToolGroup(
        identifier='builtin::websearch',
        provider_id='tavily-search',
        provider_resource_id='builtin::websearch',
        type='tool_group',
        args=None,
        mcp_endpoint=None
    ),
    ToolGroup(
        identifier='builtin::rag',
        provider_id='rag-runtime',
        provider_resource_id='builtin::rag',
        type='tool_group',
        args=None,
        mcp_endpoint=None
    )
]

In [7]:
tools = client.tools.list()
rich.print(tools)

[
    Tool(
        description='Search the web for information',
        identifier='web_search',
        parameters=[
            Parameter(
                description='The query to search for',
                name='query',
                parameter_type='string',
                required=True,
                default=None
            )
        ],
        provider_id='tavily-search',
        provider_resource_id='web_search',
        tool_host='distribution',
        toolgroup_id='builtin::websearch',
        type='tool',
        metadata=None
    ),
    Tool(
        description='Insert documents into memory',
        identifier='insert_into_memory',
        parameters=[],
        provider_id='rag-runtime',
        provider_resource_id='insert_into_memory',
        tool_host='distribution',
        toolgroup_id='builtin::rag',
        type='tool',
        metadata=None
    ),
    Tool(
        description='Search for information in a database.',
        identifier='knowledge_search',
        parameters=[
            Parameter(
                description='The query to search for. Can be a natural language sentence or keywords.',
                name='query',
                parameter_type='string',
                required=True,
                default=None
            )
        ],
        provider_id='rag-runtime',
        provider_resource_id='knowledge_search',
        tool_host='distribution',
        toolgroup_id='builtin::rag',
        type='tool',
        metadata=None
    )
]

## Baseline

In [8]:
client.toolgroups.register(
    toolgroup_id="builtin::websearch",
    provider_id="tavily-search",
)

In [9]:
toolgroups = client.toolgroups.list()
rich.print(toolgroups)

[
    ToolGroup(
        identifier='builtin::websearch',
        provider_id='tavily-search',
        provider_resource_id='builtin::websearch',
        type='tool_group',
        args=None,
        mcp_endpoint=None
    ),
    ToolGroup(
        identifier='builtin::rag',
        provider_id='rag-runtime',
        provider_resource_id='builtin::rag',
        type='tool_group',
        args=None,
        mcp_endpoint=None
    )
]

In [10]:
agent = Agent(
    client,
    model="meta-llama/Llama-3.2-3B-Instruct",
    instructions="You are a helpful assistant.",
    tools=["builtin::websearch"],
)

In [11]:
response = agent.create_turn(
    messages=[{"role": "user", "content": "Are you able to search the web?"}],
    session_id=agent.create_session(f"tool-check-{uuid.uuid4()}"),
    stream=False,
)
rich.print(response)

Turn(
    input_messages=[UserMessage(content='Are you able to search the web?', role='user', context=None)],
    output_message=CompletionMessage(
        content='Yes, I can search the web for information on a wide range of topics. What would you like me to 
look up?',
        role='assistant',
        stop_reason='end_of_turn',
        tool_calls=[]
    ),
    session_id='1a3d86cf-f8bd-4c1b-9113-1cb64a645179',
    started_at=datetime.datetime(2025, 5, 11, 3, 13, 10, 202123, tzinfo=TzInfo(UTC)),
    steps=[
        InferenceStep(
            api_model_response=CompletionMessage(
                content='Yes, I can search the web for information on a wide range of topics. What would you like 
me to look up?',
                role='assistant',
                stop_reason='end_of_turn',
                tool_calls=[]
            ),
            step_id='357d9451-5711-4d37-bee5-14a0fd619f6f',
            step_type='inference',
            turn_id='7c20cb82-51fa-4e93-9669-f553c642a392',
            completed_at=datetime.datetime(2025, 5, 11, 3, 13, 17, 591291, tzinfo=TzInfo(UTC)),
            started_at=datetime.datetime(2025, 5, 11, 3, 13, 10, 202150, tzinfo=TzInfo(UTC))
        )
    ],
    turn_id='7c20cb82-51fa-4e93-9669-f553c642a392',
    completed_at=datetime.datetime(2025, 5, 11, 3, 13, 17, 604244, tzinfo=TzInfo(UTC)),
    output_attachments=[]
)

In [16]:
agent = Agent(
    client, 
    model="meta-llama/Llama-3.2-3B-Instruct",
    instructions="You are a helpful assistant. Use websearch tool to help answer questions.",
    tools=["builtin::websearch"],
)
user_prompts = [
    # "Are there any weather-related risks in my area?",
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?",

]
session_id = agent.create_session(f"test-session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")

    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
        stream=True,
    )
    for log in AgentEventLogger().log(response):
        log.print()

User> Are there any weather-related risks in my area that could disrupt network connectivity or system 
availability?

inference> brave_search.call(query="weather-related risks near me")
tool_execution> Tool:brave_search Args:{'query': 'weather-related risks near me'}
tool_execution> Tool:brave_search Response:{"query": "weather-related risks near me", "top_k": [{"title": "Weather in current location", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Current', 'region': 'Harbour Island', 'country': 'Bahamas', 'lat': 25.4333, 'lon': -76.7833, 'tz_id': 'America/Nassau', 'localtime_epoch': 1746933435, 'localtime': '2025-05-10 23:17'}, 'current': {'last_updated_epoch': 1746933300, 'last_updated': '2025-05-10 23:15', 'temp_c': 24.8, 'temp_f': 76.6, 'is_day': 0, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 17.0, 'wind_kph': 27.4, 'wind_degree': 100, 'wind_dir': 'E', 'pressure_mb': 1020.0, 'pressure_in': 30.11, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 85, 'cloud': 28, 'feelslike_c': 27.1, 'feelslike

## Prompt Chaining

In [17]:
agent = Agent(
    client,
    model="meta-llama/Llama-3.2-3B-Instruct",
    instructions="""You are a helpful assistant. 
    When a user asks about their location, use the get_location tool. 
    When searching for nearby places, use the websearch tool.
    """,
    tools=[get_location, "builtin::websearch"],
)
user_prompts = [
    "Where am I?",
    "Are there any weather-related risks in my area?",
]
session_id = agent.create_session(f"prompt-chaining-session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()

User> Where am I?

inference> 

[get_location(query="my current location")]
tool_execution> Tool:get_location Args:{'query': 'my current location'}
tool_execution> Tool:get_location Response:"Your current location is: Toronto, Ontario, CA"
inference> {"location": "Toronto, Ontario, CA"}<|python_tag|>{"location": "Toronto, Ontario, CA"}


User> Are there any weather-related risks in my area?

inference> [websearch(query="weather forecast for Toronto, Ontario, CA")]<|python_tag|>[websearch(query="weather forecast for Toronto, Ontario, CA")]


### ReAct Agent 

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

For example, when asked "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent:
1. **Reasons** it needs location information first
2. **Acts** by calling the `get_location` client tool
3. **Observes** the location result
4. **Reasons** about the next step
5. **Acts** by calling the `websearch` tool with observed location
6. **Observes** and processes the search results

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

In [ ]:
# this fails, gotta update ollama
agent = ReActAgent(
    client=client,
    model="meta-llama/Llama-3.2-3B-Instruct",
    tools=[get_location, "builtin::websearch"],
    # response_format={
    #     "type": "json_schema",
    #     "json_schema": ReActOutput.model_json_schema(),
    # },
)
user_prompts = [
    "Are there any weather-related risks in my area?",
]
session_id = agent.create_session(f"web-session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()